### Riddler Classic: 

From Fred Blundun comes a simple game of definitive diffidence:

Martina and Olivia each secretly generate their own random real number, selected uniformly between 0 and 1. Starting with Martina, they take turns declaring (so the other can hear) who they think probably has the greater number until the first moment they agree. Throughout this process, their respective numbers do not change. So for example, their dialogue might go as follows:

```
Martina: My number is probably bigger.

Olivia: My number is probably bigger.

Martina: My number is probably bigger.

Olivia: My number is probably bigger.

Martina: Olivia’s number is probably bigger.

```
They are playing as a team, hoping to maximize the chances they correctly predict who has the greater number.

For any given round with randomly generated numbers, what is the probability that the person they agree on really does have the bigger number?


### Initial Thoughts:

Given the comment on maximizing chances, it makes me wonder if some kind of strategy can be employed where their likelihood of saying "my number is probably bigger" is associated with the value of their number. 

E.g. If my number is 0.05 I doubt my number is bigger, however if my number is 0.99 I am pretty confident my number is bigger.

Need to model out a confidence value, which I think could be used in conjunction with the sequence of the conversation:
- `n` = max sequence size, dictated by how precise we want to be (.1 increments use 10, .01 use 100, etc.).
- In each round each user's comment will represent a step in our sequence to n, at which point their is a tie of 1?

In [1]:
import numpy as np
from random import random

In [2]:
def confidenceSeq(floatVal, precision = 1):
    """Input users float & expected precision, return sequence at which they lose confidence"""
    return round(floatVal * 10**precision, 0)

In [3]:
# low precision
test_arr = np.asarray([0.01,0.3,0.5,0.7,0.99])
[confidenceSeq(i) for i in test_arr]

[0.0, 3.0, 5.0, 7.0, 10.0]

In [4]:
# higher precision
test_arr = np.asarray([0.01,0.3,0.5,0.7,0.99])
[confidenceSeq(i, 3) for i in test_arr]

[10.0, 300.0, 500.0, 700.0, 990.0]

In [5]:
step = 0
m_val = random() # m goes first
o_val = random()

conf_vals = [confidenceSeq(i, 2) for i in [m_val, o_val]]
print(f"Martina has a value of {m_val:.4f}, Olivia has a value of {o_val:.4f}")

while True:
    
    step += 1
    
    if step % 2 == 0:
        print("O's turn")
        if step > conf_vals[1]:
            print("M is bigger")
            break
    else:
        print("M's turn")
        if step > conf_vals[0]:
            print(" is bigger")
            break        

Martina has a value of 0.4350, Olivia has a value of 0.5335
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
O's turn
M's turn
 is bigger


### Combining Into a Simulation: 

In [6]:
#
win = 0
games = 100
missed_wins = {}

for game in range(games):

    # new game run
    step = 0
    m_val = random() # m goes first
    o_val = random()
    game_vals = [m_val, o_val]
    conf_vals = [confidenceSeq(i, 2) for i in game_vals]
    true_winner_idx = game_vals.index(max(game_vals))
    name_list = ["Martina", "Olivia"]
    #print(f"Martina has a value of {game_vals[0]:.4f}, Olivia has a value of {game_vals[1]:.4f}")
    #print(f"True winner should be: {name_list[true_winner_idx]}")

    while True:

        step += 1

        if step % 2 == 0:
            if step > conf_vals[1]:
                pred_winner_idx = 0 
                break
        else:
            if step > conf_vals[0]:
                pred_winner_idx = 1
                break

    # Determine performance: Store off misses for review 
    if pred_winner_idx == true_winner_idx:
        win += 1
    else:
        missed_wins[game] = game_vals
        
### Performance metrics
print(f"For {games} games there was an accuracy of {100 * win / games:.3f}%")

For 100 games there was an accuracy of 98.000%


In [7]:
# makes sense that our miss will occur when very close....as we expand precision this shouldn't happen....which is next.
missed_wins

{21: [0.7112954155843861, 0.7120397239398537],
 97: [0.5796914882856266, 0.5764447046350438]}

### Many Sims: 

We get closer & closer to 100%. 

In [8]:
for precision in [0, 1,2,3,4]:
    win = 0
    games = 100_000 

    for game in range(games):

        # new game run
        step = 0
        m_val = random() # m goes first
        o_val = random()
        game_vals = [m_val, o_val]
        conf_vals = [confidenceSeq(i, precision) for i in game_vals]
        true_winner_idx = game_vals.index(max(game_vals))
        name_list = ["Martina", "Olivia"]
        #print(f"Martina has a value of {game_vals[0]:.4f}, Olivia has a value of {game_vals[1]:.4f}")
        #print(f"True winner should be: {name_list[true_winner_idx]}")

        while True:

            step += 1

            if step % 2 == 0:
                if step > conf_vals[1]:
                    pred_winner_idx = 0 
                    break
            else:
                if step > conf_vals[0]:
                    pred_winner_idx = 1
                    break

        # Determine performance: Store off misses for review 
        if pred_winner_idx == true_winner_idx:
            win += 1

    ### Performance metrics
    print(f"For {games} games with precision {precision} there was an accuracy of {100 * win / games:.3f}%")

For 100000 games with precision 0 there was an accuracy of 74.958%
For 100000 games with precision 1 there was an accuracy of 95.306%
For 100000 games with precision 2 there was an accuracy of 99.543%
For 100000 games with precision 3 there was an accuracy of 99.949%
For 100000 games with precision 4 there was an accuracy of 99.988%


### But, The Inner-Game Simulations Aren't Actually Needed:

- We can very quickly establish the winner each game based on the min value
- This allows us to quickly increase precision & game size

In [9]:
for precision in [0, 1,2,3,4,5,6,7,8,9]:
    win = 0
    games = 1_000_000 

    for game in range(games):

        # new game run
        m_val = random() # m goes first
        o_val = random()
        game_vals = [m_val, o_val]
        conf_vals = [confidenceSeq(i, precision) for i in game_vals]
        true_winner_idx = game_vals.index(max(game_vals))
        start_step = min(conf_vals) - 1

        while True:

            start_step += 1

            if start_step % 2 == 0:
                if start_step > conf_vals[1]:
                    pred_winner_idx = 0 
                    break
            else:
                if start_step > conf_vals[0]:
                    pred_winner_idx = 1
                    break


        # Determine performance: Store off misses for review 
        if pred_winner_idx == true_winner_idx:
            win += 1

    ### Performance metrics
    print(f"For {games} games with precision {precision} there was an accuracy of {100 * win / games:.3f}%")

For 1000000 games with precision 0 there was an accuracy of 75.011%
For 1000000 games with precision 1 there was an accuracy of 95.227%
For 1000000 games with precision 2 there was an accuracy of 99.507%
For 1000000 games with precision 3 there was an accuracy of 99.951%
For 1000000 games with precision 4 there was an accuracy of 99.995%
For 1000000 games with precision 5 there was an accuracy of 100.000%
For 1000000 games with precision 6 there was an accuracy of 100.000%
For 1000000 games with precision 7 there was an accuracy of 100.000%
For 1000000 games with precision 8 there was an accuracy of 100.000%
For 1000000 games with precision 9 there was an accuracy of 100.000%
